In [58]:
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Click the "Load More" button for scraping more news

In [59]:
def extendPage():
    while True:
        try:
            try:
                # Just to locate to the Load More button to be clicked
                load_more = driver.find_element_by_class_name('quicklinks')
                actions = ActionChains(driver)
                actions.move_to_element(load_more).perform()
            except NoSuchElementException:
                print('Skip')
                break

            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "loadMorestories"))).click()
            break
        except ElementClickInterceptedException as a:
            print("Try again")
            pass
        except TimeoutException as b:
            print("Try again")
            pass
        except NoSuchElementException as c:
            print("Try again")
            pass

### Iterate the given list to text or url

In [60]:
def toList(given_list, condition):
    convert = []
    for i in given_list:
        if condition == 'link':
            convert.append(i.get_attribute('href'))
        else:
            convert.append(i.text)
    non_empty_ele = [j for j in convert if j]
    return non_empty_ele

### Activate chrome driver

In [61]:
driver = webdriver.Chrome()
driver.get('https://www.thestar.com.my/')

### Extract all types of news

In [62]:
types = driver.find_elements_by_xpath(".//li[@class = 'dropdown']")
all_type = toList(types, 'text')
all_type.remove('Videos')
all_type.remove('Photos')
print(all_type)

['StarPlus', 'News', 'Asean+', 'Business', 'Sport', 'Metro', 'Lifestyle', 'Food', 'Tech', 'Education', 'Opinion', 'StarPicks']


In [63]:
links_list = []
news_list = []

### Browse through every type of webpage, scrape the links and store into a list


In [64]:
for element in all_type:
    xpath = f"//a[contains(text(), '{element}')]"
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath))).click()
    print(xpath)
    extendPage()

    travel1 = driver.find_elements_by_xpath(".//main//div[5]//h2/a")
    if element.lower() == 'starplus' or element.lower() == 'business' or element.lower() == 'lifestyle' or element.lower() == 'opinion':
        travel2 = driver.find_elements_by_xpath(".//main//div[6]//h2/a")
    else:
        travel2 = driver.find_elements_by_xpath(".//main//div[6]/div/div/div[2]//h2/a")

    links_list1 = toList(travel1, 'link')
    links_list2 = toList(travel2, 'link')
    links_list = links_list + links_list1 + links_list2
    print(len(links_list))

driver.close()

//a[contains(text(), 'StarPlus')]
Skip
17
//a[contains(text(), 'News')]
57
//a[contains(text(), 'Asean+')]
Skip
87
//a[contains(text(), 'Business')]
127
//a[contains(text(), 'Sport')]
167
//a[contains(text(), 'Metro')]
203
//a[contains(text(), 'Lifestyle')]
Skip
239
//a[contains(text(), 'Food')]
269
//a[contains(text(), 'Tech')]
Skip
299
//a[contains(text(), 'Education')]
Skip
313
//a[contains(text(), 'Opinion')]
Skip
322
//a[contains(text(), 'StarPicks')]
Skip
339


### Remove duplicates from list


In [65]:
filtered_links = []
[filtered_links.append(x) for x in links_list if x not in filtered_links]
print(f'No. of non-duplicated link: {len(filtered_links)}')

No. of non-duplicated link: 294


### Considering the Food category contains some recipes which will lead to another web page, therefore I decided to remove non The Star link here


In [66]:
for i in range(5):
    for i, ele in enumerate(filtered_links):
        if not ele.startswith('https://www.thestar.com.my/'):
            del filtered_links[i]
print(f'No. of non-TheStar link: {len(filtered_links)}')
filtered_links

No. of non-TheStar link: 288


['https://www.thestar.com.my/news/nation/2021/04/24/interactive-how-malaysias-fight-against-covid-19-is-progressing-updated-weekly',
 'https://www.thestar.com.my/news/nation/2020/01/26/the-rapid-global-spread-of-the-novel-coronavirus---total-cases-so-far',
 'https://www.thestar.com.my/news/nation/2021/06/02/interactive-more-covid-19-patients-are-being-brought-in-dead---why-this-happens-and-what-needs-to-be-done',
 'https://www.thestar.com.my/news/nation/2021/05/25/interactive-malaysian-twitter-users-rank-near-the-top-globally-in-support-of-palestine',
 'https://www.thestar.com.my/news/nation/2021/05/23/interactive-suffering-in-silence',
 'https://www.thestar.com.my/news/nation/2020/03/23/covid-19-current-situation-in-malaysia-updated-daily',
 'https://www.thestar.com.my/news/nation/2021/05/16/interactive-teachers-a-class-above-in-difficult-times',
 'https://www.thestar.com.my/news/nation/2021/05/09/interactive-more-policies-needed-to-overcome-unemployment-among-young-m039sian-women',
 

### Browse through each url and crawl the news content

In [ ]:
for link in filtered_links:
    driver = webdriver.Chrome()
    url = f"{link}"
    print(link)
    driver.get(url)

    try:
        title = driver.find_element_by_tag_name('h1').text
    except NoSuchElementException:
        driver.close()
        continue

    try:
        author = driver.find_elements_by_xpath(".//div[@class = 'credit__authors']/a")
        author = toList(author, 'text')
        author = re.sub(",", " ", '，'.join(author))
    except NoSuchElementException:
        author = ''
        pass

    try:
        date = driver.find_element_by_xpath(".//p[@class = 'date']").text
    except NoSuchElementException:
        date = ''
        pass

    try:
        timestamp = driver.find_element_by_xpath(".//time[@class = 'timestamp']").text
    except NoSuchElementException:
        timestamp = ''
        pass

    # Extract all the text of the news content
    parent = driver.find_element_by_id('story-body')
    content = list(parent.text.split('\n'))

    # The following 4 statements are to extract all irrelevant content such as chart and image caption
    chart_text = parent.find_elements_by_xpath(".//div[contains(@class, 'flourish')]")
    inline_text = parent.find_elements_by_xpath(".//span[contains(@class, 'inline-caption')]")
    caption_text = parent.find_elements_by_xpath(".//p[contains(@class, 'caption')]")
    extra_text = chart_text + inline_text + caption_text

    full_content = toList(extra_text, 'text')
    full_content = list(set(full_content))

    # Filter out irrelevant content
    for index, i in enumerate(content):
        for j in full_content:
            if i == j:
                content[index] = ''

    # Remove the enter and empty field from the extracted news content
    real_content = ''
    for element in content:
        real_content += element

    # Determine the type of the news by using the url
    for category in all_type:
        str = f"https://www.thestar.com.my/{category.lower()}"
        result = re.search(str, link)
        if result is not None:
            break

    print(date)
    print(timestamp)
    print(title)
    print(category)
    print(author)
    print(real_content)

    news_items = {'Date': date, 'Time': timestamp, 'Type': category, 'Author': author, 'Title': title, 'Content': real_content}
    news_list.append(news_items)
    print(len(news_list))
    driver.close()
    time.sleep(1)

https://www.thestar.com.my/news/nation/2021/04/24/interactive-how-malaysias-fight-against-covid-19-is-progressing-updated-weekly
Saturday, 05 Jun 2021
7:00 AM MYT
INTERACTIVE: How Malaysia’s fight against Covid-19 is progressing (updated weekly)
News
RAZAK AHMAD
PETALING JAYA: A total of 3,330,436 Covid-19 vaccine doses have been administered under the country’s National Covid-19 Immunisation Programme as at June 3.The vaccination rate has been rising for the past few weeks after getting off to a slow start and on May 27 the number of jabs given daily breached the 100,000 mark, with 107,340 doses administered.The vaccination programme started on Feb 24 with a target of covering at least 80% of the country’s adult population.The vaccination programme is being carried out in three phases.The first covers frontliners.The second which began on April 19 involves high-risk groups and those aged 60 and above.The rest of the Malaysian population aged 18 years and above will be vaccinated in th

https://www.thestar.com.my/news/nation/2021/05/25/interactive-malaysian-twitter-users-rank-near-the-top-globally-in-support-of-palestine
Tuesday, 25 May 2021
7:00 AM MYT
INTERACTIVE: Malaysian Twitter users rank near the top globally in support of Palestine
News
DIYANA PFORDTEN，RAZAK AHMAD
PETALING JAYA: Computer science graduate Muadz Halim normally tweets about football and Japanese manga, but when Israel attacked Palestine earlier this month, his focus expanded.The 23-year-old from Perak began to retweet and use pro-Palestine hashtags from early May, at a rate of 80 to 120 times each day."I hope that by doing this, I can at least raise awareness among my social media followers that the Palestinians are suffering and that we need to always support and show them solidarity."My greatest hope is that Palestine will be free and that there will be no more suffering,” said Muadz, who has racked up a total of 2,707 mentions and retweets on the topic.He is one of an army of local social medi

https://www.thestar.com.my/news/nation/2020/03/23/covid-19-current-situation-in-malaysia-updated-daily
Thursday, 22 Apr 2021
5:30 PM MYT
Covid-19: Cases up by 2,875 bringing total to 384,688 (updated daily)
News
DIYANA PFORDTEN，RAZAK AHMAD
PETALING JAYA: Malaysia announced its first Covid-19 cases on Jan 25 involving three China tourists who had entered Malaysia via Johor from Singapore on Jan 23.The number of cases then rose to 22 by Feb 16, representing a first wave of cases.A second wave of cases began on Feb 27 and since then the number of people affected by the coronavirus has risen to more than 1,000.Malaysia is now facing a third wave of infections which began on Sept 20.(Click here for more details on the latest global Covid-19 figures)Since May 30, the Health Ministry has released daily district-level data on the total number of new cases recorded within a two-week period.Find out the locations and trained personnel permitted to do the screenings in your area in the search box

https://www.thestar.com.my/news/nation/2021/05/06/interactive-vaccine-passports---the-what-how-why-and-why-not
Thursday, 06 May 2021
7:00 AM MYT
INTERACTIVE: Vaccine passports – the what, how, why and why not
News
DIYANA PFORDTEN
PETALING JAYA: Countries around the world are exploring ways to show proof of an individual's Covid-19 immunity in order to allow travel and other easing of social restrictions.Several countries have initiated travel passports, which are Covid-19 vaccine certificates or any physical or digital document that would prove you've received both doses (or one dose for certain vaccines) of any of the available Covid-19 vaccines.In some countries, vaccine certificates can also be issued to those who have recently tested negative for Covid-19 or have built immunity against it after having been infected.All this would allow citizens to travel among participating countries or be granted exemptions to certain Covid-19 restrictions in their own country.A recent survey by m

https://www.thestar.com.my/news/nation/2021/04/24/interactive-myanmar-at-a-crossroads
Saturday, 24 Apr 2021

INTERACTIVE: Myanmar at a Crossroads
News

Myanmar at a Crossroadsby MAZWIN NIK ANIS and JOSEPH KAOS, JRAP PhotoWHEN the 10 members of Asean meet for a special summit in Jakarta today, the most urgent and immediate task at hand is to resolve the political crisis in Myanmar, which has so far seen over 700 people killed in protests across the country.The fact that the summit, the first physical meeting amidst a pandemic, is taking place speaks volume of the urgent matter at hand and will see the attendance of heads of government, except for Thai Prime Minister Prayuth Chan o-cha who has confirmed that he will be skipping the meet.Thailand will instead be represented by Deputy Prime Minister and Foreign Minister Don Pramudwinai.If the head of Myanmar’s military government, Senior General Min Aung Hlaing, does indeed show up for the meet, this will be his first known foreign trip si

...

https://www.thestar.com.my/news/nation/2021/06/08/hadi-awang-in-stable-condition-says-press-sec
Tuesday, 08 Jun 2021
5:29 PM MYT
Hadi Awang in stable condition, says press sec
News

KUALA TERENGGANU (Bernama): PAS president Datuk Seri Abdul Hadi Awang who is currently being treated at the National Heart Institute (IJN) is in stable condition.This was confirmed by his press secretary Abdul Malik Ab Razak, when contacted by Bernama Tuesday (June 8).Abdul Malik also denied rumours on Abdul Hadi’s health condition saying that the Prime Minister’s special envoy to the Middle East is unwell and medically unfit to manage the party’s affairs."It (rumours on Abdul Hadi’s health) is not true. Tuan Guru (Abdul Hadi) is in stable condition. Any information regarding Tuan Guru’s health can be referred to the official statement on his official Facebook page, ” he said.Last Sunday (June 6), Batu Buruk assemblyman Muhammad Khalil, who is also Abdul Hadi’s son, said his father had been admitted to IJN 

https://www.thestar.com.my/news/nation/2021/06/08/essential-trade-sector-services-allowed-during-lockdown-already-made-clear-in-may-31-statement-says-minister
Tuesday, 08 Jun 2021
3:55 PM MYT
Essential trade sector services allowed during lockdown already made clear in May 31 statement, says minister
News
RAHIMY RAHIM
PETALING JAYA: Only breweries and shops focused on liquor are not allowed to operate during the current movement control order, says Datuk Seri Alexander Nanta Linggi.The Domestic Trade and Consumer Affairs Minister dismissed claims by Bangi MP Dr Ong Kian Ming who claimed that the ministry remained silent on several issues including the sale of alcohol and permission to sell cigarettes."It must be stressed here that stores selling only liquor products are not allowed to operate."The sale of cigarettes in convenience stores is allowed as the stores have been listed as essential," he said in a statement on Tuesday (June 8).Last week, there was some confusion as to whether 

https://www.thestar.com.my/news/nation/2021/06/08/govt-should-explain-high-covid-19-positivity-rate-despite-reduction-in-number-of-screenings-says-najib
Tuesday, 08 Jun 2021
2:03 PM MYT
Govt should explain high Covid-19 positivity rate despite reduction in number of screenings, says Najib
News
RAHIMY RAHIM
PETALING JAYA: The government should explain why the number of Covid-19 screenings had gone down but the positivity rate of Covid-19 remains high, says Datuk Seri Najib Razak.The former prime minister said he noticed that the number of Covid-19 screenings had fallen consistently, from 125,000 to 80,000 on Monday (June 8).“Despite all of this, the positivity rate (the total positive cases compared to the total screenings), the number of intensive care unit (ICU) patients and the number of new clusters detected daily remain consistently high, ” he said in a Facebook post on Tuesday (June 8).Najib said he hoped the government could explain this.The government said last week that it was 

https://www.thestar.com.my/news/nation/2021/06/08/eight-detained-in-connection-with-prostitution-syndicate-in-kl
Tuesday, 08 Jun 2021
12:04 PM MYT
Eight detained in connection with prostitution syndicate in KL
News
FARIK ZOLKEPLI
KUALA LUMPUR: Four foreign women were among eight individuals detained following raids at an apartment in Jalan Gombak and a hotel in Jalan Genting Klang.Those detained were connected to a prostitution ring, which offered its services online.Kuala Lumpur CID chief Senior Asst Comm Saiful Annuar Yusoff said police raided both premises at about 10.40pm on Monday (June 7)."We detained two Vietnamese women, a Thai woman, a female China national, along with a local woman and three local men."Those detained were aged between 20 and 46," he said when contacted on Tuesday (June 8).Among the items seized were two laptops, four computers, 19 mobile phones and RM1,117 cash, he added."We believe they have been promoting sex services online via WhatsApp, WeChat and Telegra

Tuesday, 08 Jun 2021

Retired couple travel 127km to get their shots
News
REMAR NORDIN
JOHOR BARU: An elderly couple drove for almost two hours from Batu Pahat to get their AstraZeneca shots at the mega vaccination centre (PPV) at the Persada Interna-tional Convention Centre here.Retired school principal Idris Ahmad, 67, and his wife Azmah Hairun, 64, were determined to be vaccinated so they went on a 127km journey from their home in Batu Pahat.Under the ongoing national lockdown, inter-district and interstate travel are allowed for people who have vaccination appointments.Idris said they left their home after subuh (dawn) prayers just to be at the PPV in time.“There were not many vehicles along the highway due to the movement control order. It was a nice, slow drive.“Both of us had looked forward to getting the vaccine, ” he said at the PPV here yesterday.Idris said he was very happy that both he and his wife got the same appointment dates when their children registered their names th

### Store the crawled data into data frame

In [ ]:
current = pd.DataFrame(news_list)
current.head()

In [ ]:
print("Shape    : ", current.shape)

### Read older csv file

In [ ]:
before = pd.read_csv('News.csv')
before.head()

In [ ]:
print("Shape    : ", before.shape)

### Combine 2 data frames

In [ ]:
# Concatenate current and older data
data = pd.concat([before, current])
# Drop duplicates between previous and current data
data.drop_duplicates(inplace = True, subset = ['Content'])
# Store the data back to the News csv file
data.to_csv('News.csv', index=False)
data.head()

In [ ]:
print("Rows     : ", data.shape[0])
print("Columns  : ", data.shape[1])
print("Shape    : ", data.shape)
print("Features : ", data.columns.tolist())